# Group Data Analysis | DATA 5600
## Simple Linear Regression
### Owen Mendiola & Carter Daly

## Introduction

The dataset used for this analysis was obtained from the Spotify API. We got information from the top 2500 songs of all time. This was based off of total streams. Key metrics in the dataset include blah blah blah blah blah. 
For this analysis, our primar focus is seeing if we can predict how popular a song is _________

## Exploratory Data Analysis

# edit this version of my groupproject 1 
Before performing any modeling, we want to explore the data and see if we can see any underlying structures/relationships. Again, we will be looking at the relationship between Caffeine Intake (in mg) and Sleep Quality Score

In [ ]:
%pip install pandas numpy seaborn matplotlib statsmodels mlxtend scikit-learn

In [22]:
# Put the imports we need here also read in the file
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.linear_model import LinearRegression

from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

# Set seed
import random
random.seed(5600)


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.9.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached statsmodels-0.14.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached mlxtend-0.23.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached contourpy-1.3.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.54.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (163 kB)
  Using cached kiwisolver-1.4.7-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pillow-11.0.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached patsy-0.5.6-py2.py3-none-any.whl.metadata (3.5 kB)
  Using c

In [23]:
df = pd.read_csv('SongOutputs.csv')
df.head()

,name,artist,album,release_date,popularity,spotify_url,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,Streams,Daily
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,88,https://open.spotify.com/track/0VjIjW4GlUZAMYd...,0.513,0.730,1.0,-5.940,1.0,0.0598,0.00143,0.000095,0.0897,0.334,171.001,"4,480,488,178","1,856,931"
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,85,https://open.spotify.com/track/7qiZfU4dY1lWllz...,0.825,0.652,1.0,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,"4,028,229,987","1,541,000"
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,87,https://open.spotify.com/track/7qEHsqek33rTcFN...,0.501,0.405,1.0,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,"3,605,278,930","1,633,860"
3,As It Was,Harry Styles,Harry's House,2022-05-20,88,https://open.spotify.com/track/4Dvkj6JhhA12EX0...,0.520,0.731,6.0,-5.338,0.0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,"3,560,182,786","1,633,094"
4,Starboy,The Weeknd,Starboy,2016-11-25,88,https://open.spotify.com/track/7MXVkk9YMctZqd1...,0.679,0.587,7.0,-7.015,1.0,0.2760,0.14100,0.000006,0.1370,0.486,186.003,"3,499,250,243","1,948,505"


Before we continue we need to remove certain variables like name, artist, popularity, data, etc. These don't give any useful information to our analysis. We only care about our continours/categorical predictors and our outcome variable Daily

In [ ]:
# Create a new DataFrame excluding specified columns
df2 = df.drop(columns=['name', 'artist', 'album', 'release_date', 'popularity', 'spotify_url', 'Streams'])
# Clean the 'Daily' column by removing commas and converting to numeric
df2['Daily'] = df2['Daily'].replace({',': ''}, regex=True).astype(float)
# Remove rows with any null or empty values
df2 = df2.dropna()
print(df2.head())

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.513   0.730  1.0    -5.940   1.0       0.0598       0.00143   
1         0.825   0.652  1.0    -3.183   0.0       0.0802       0.58100   
2         0.501   0.405  1.0    -5.679   1.0       0.0319       0.75100   
3         0.520   0.731  6.0    -5.338   0.0       0.0557       0.34200   
4         0.679   0.587  7.0    -7.015   1.0       0.2760       0.14100   

   instrumentalness  liveness  valence    tempo      Daily  
0          0.000095    0.0897    0.334  171.001  1856931.0  
1          0.000000    0.0931    0.931   95.977  1541000.0  
2          0.000000    0.1050    0.446  109.891  1633860.0  
3          0.001010    0.3110    0.662  173.930  1633094.0  
4          0.000006    0.1370    0.486  186.003  1948505.0  


In [ ]:
df2.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,Daily
count,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2.016000e+03
mean,0.644258,0.647750,5.256944,-6.551891,0.603175,0.086663,0.220530,0.022319,0.170430,0.506947,121.250674,5.484831e+05
std,0.148120,0.182514,3.578049,2.849308,0.489361,0.083903,0.247967,0.110326,0.130439,0.230982,28.597777,5.218727e+05
min,0.150000,0.016700,0.000000,-26.813000,0.000000,0.022700,0.000008,0.000000,0.023200,0.035200,48.718000,3.180900e+04
25%,0.548000,0.534000,2.000000,-7.859000,0.000000,0.036600,0.027775,0.000000,0.091400,0.328000,98.503750,2.521300e+05
50%,0.658000,0.670000,5.000000,-5.972500,1.000000,0.052350,0.119500,0.000002,0.118000,0.499500,119.943500,4.036090e+05
75%,0.753000,0.785250,8.000000,-4.672500,1.000000,0.095050,0.331250,0.000164,0.205000,0.689000,139.076750,6.961425e+05
max,0.965000,0.988000,11.000000,-1.190000,1.000000,0.592000,0.989000,0.961000,0.914000,0.981000,207.970000,7.944731e+06


In [ ]:
# code for scatterplot matrix

In [ ]:
# code for correlation matrix

In [ ]:
# color coded correlation matrix

## Variable Selection

In [ ]:
area to explain why we do variable selection and what steps we will do 

### Backward Selection

In [ ]:
# spot to perform backward selection 

In [ ]:
# show the line graph 

In [ ]:
# table of top models

### Variable Selection Conclusion

We decided to pick these variables ________

## Multiple Linear Regression

what are we doing here _____. We first have to check assumptions before we do anything else